# Tuning Parameters for Neural Network Models
MSc in Statistical Science\
University of Oxford\
Group-assessed practical\
HT 2024

## Based on file `NN,LDA+NN,PCA+NN.ipynb`

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from search_param.grid_search import read_data, grid_search, rand_search, pipeline_search
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from keras_tuner import RandomSearch
import shutil
import json

## Load dataset

In [2]:
X_train, X_val, y_train, y_val = read_data()

## Neural Network

In [3]:
# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Fit label encoder and return encoded labels
y_train_encoded = label_encoder.fit_transform(y_train)
y_val_encoded = label_encoder.transform(y_val)

# Convert labels to one-hot encoding
y_train_onehot = to_categorical(y_train_encoded)
y_val_onehot = to_categorical(y_val_encoded)

### Standard scaling

In [4]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train_sc = scaler.transform(X_train)
X_val_sc = scaler.transform(X_val)

In [5]:
def build_model(hp):
    model = Sequential()
    model.add(Dense(units=hp.Int('units', min_value=32, max_value=518, step=32),
                    activation=hp.Choice('activation', values=['relu', 'softplus', 'tanh', 'sigmoid']),
                    input_shape=(518,)))  # Ensure this matches your feature size
    # Use hp.Choice to select the dropout rate
    model.add(Dropout(rate=hp.Float('dropout_0', min_value=0.0, max_value=0.5, step=0.05)))

    for i in range(hp.Int('layers', 1, 5)):
        model.add(Dense(units=hp.Int(f'units_{i}', min_value=32, max_value=518, step=32),
                        activation=hp.Choice(f'activation_{i}', values=['relu', 'softplus', 'tanh', 'sigmoid'])))
        # Use hp.Choice for each layer's dropout rate
        model.add(Dropout(rate=hp.Float(f'dropout_{i+1}',  min_value=0.0, max_value=0.5, step=0.05)))

    model.add(Dense(8, activation='softmax'))  # Adjust the number of units based on your number of classes
    model.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['accuracy'])
    return model


# Be careful with this operation to avoid deleting important data
shutil.rmtree('my_dir/hparam_tuning')

tuner_sc = RandomSearch(
    build_model,
    objective='val_accuracy',
    executions_per_trial=1,  # Increase for more robust results
    directory='my_dir',
    project_name='hparam_tuning'
)

c:\Users\Liu_h\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [6]:
# Use a lower number of epochs for the search phase
tuner_sc.search(X_train_sc, y_train_onehot, epochs=20, validation_data=(X_val_sc, y_val_onehot))

Trial 10 Complete [00h 00m 18s]
val_accuracy: 0.550000011920929

Best val_accuracy So Far: 0.60916668176651
Total elapsed time: 00h 04m 04s


In [7]:
# Get the best model
sc_best = tuner_sc.get_best_models(num_models=1)[0]
sc_best.summary()

c:\Users\Liu_h\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 480)            │       249,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 480)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 224)            │       107,744 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 224)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 288)            │        64,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 288)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 8)              │         2,312 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 423,976 (1.62 MB)

 Trainable params: 423,976 (1.62 MB)

 Non-trainable params: 0 (0.00 B)

In [8]:
# Save the best model to a file
sc_param = sc_best.get_config()
with open('search_nn/sc_param.json', 'w') as f:
    json.dump(sc_param, f)

In [9]:
for layer in sc_best.layers:
    config = layer.get_config()  # Get the layer's configuration dict
    # The 'activation' key in the config dict contains the activation function name
    if 'activation' in config:
        print(f"Layer: {layer.name}, Activation Function: {config['activation']}")
    else:
        print(f"Layer: {layer.name}, No activation function")


Layer: dense, Activation Function: softplus
Layer: dropout, No activation function
Layer: dense_1, Activation Function: softplus
Layer: dropout_1, No activation function
Layer: dense_2, Activation Function: softplus
Layer: dropout_2, No activation function
Layer: dense_3, Activation Function: softmax


In [10]:
for layer in sc_best.layers:
    config = layer.get_config()  # Extract the layer configuration as a dictionary
    layer_type = config['name'].split('_')[0]  # Get the type of layer (e.g., "dense", "dropout")
    
    # Print layer type and configuration
    print(f"Layer Type: {layer_type.upper()}")
    if layer_type == 'dense':
        print(f"  - Units: {config['units']}")
        print(f"  - Activation: {config['activation']}")
    elif layer_type == 'dropout':
        print(f"  - Rate: {config['rate']}")

Layer Type: DENSE
  - Units: 480
  - Activation: softplus
Layer Type: DROPOUT
  - Rate: 0.4
Layer Type: DENSE
  - Units: 224
  - Activation: softplus
Layer Type: DROPOUT
  - Rate: 0.35000000000000003
Layer Type: DENSE
  - Units: 288
  - Activation: softplus
Layer Type: DROPOUT
  - Rate: 0.0
Layer Type: DENSE
  - Units: 8
  - Activation: softmax


In [11]:
test_loss, test_acc = sc_best.evaluate(X_val_sc, y_val_onehot)
print(f"Test Accuracy: {test_acc:.4f}")

38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6466 - loss: 1.2484 
Test Accuracy: 0.6092


### LDA

In [12]:
# Apply LDA for dimensionality reduction
lda = LinearDiscriminantAnalysis(n_components=None)  # n_components=None for using the maximum number of components less than the number of classes
X_train_lda = lda.fit_transform(X_train, y_train)
X_val_lda = lda.transform(X_val)

In [13]:
def build_model(hp):
    model = Sequential()
    # Dynamically set the input shape based on LDA's output
    input_shape = (X_train_lda.shape[1],)  # Use the feature size from LDA transformation

    model.add(Dense(units=hp.Int('units', min_value=32, max_value=518, step=32),
                    activation=hp.Choice('activation', values=['relu', 'softplus', 'tanh', 'sigmoid']),
                    input_shape=input_shape))  # Adjusted to the dynamic input shape

    model.add(Dropout(rate=hp.Float('dropout_0', min_value=0.0, max_value=0.5, step=0.05)))

    for i in range(hp.Int('layers', 1, 5)):
        model.add(Dense(units=hp.Int(f'units_{i}', min_value=32, max_value=518, step=32),
                        activation=hp.Choice(f'activation_{i}', values=['relu', 'softplus', 'tanh', 'sigmoid'])))
        model.add(Dropout(rate=hp.Float(f'dropout_{i+1}',  min_value=0.0, max_value=0.5, step=0.05)))

    model.add(Dense(8, activation='softmax'))  # Assuming 8 classes, adjust as necessary
    model.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['accuracy'])
    return model


# Be careful with this operation to avoid deleting important data
shutil.rmtree('my_dir/hparam_tuning')

# Now, create a new tuner instance as before
tuner_lda = RandomSearch(
    build_model,
    objective='val_accuracy',
    executions_per_trial=1,
    directory='my_dir',
    project_name='hparam_tuning'  # The same project name can be reused after deletion
)

c:\Users\Liu_h\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [14]:
# Use a lower number of epochs for the search phase
tuner_lda.search(X_train_lda, y_train_onehot, epochs=20, validation_data=(X_val_lda, y_val_onehot))

Trial 10 Complete [00h 00m 12s]
val_accuracy: 0.5558333396911621

Best val_accuracy So Far: 0.5625
Total elapsed time: 00h 02m 16s


In [15]:
# Get the best model
lda_best = tuner_lda.get_best_models(num_models=1)[0]
lda_best.summary()

c:\Users\Liu_h\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 26 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 480)            │         3,840 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 480)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 96)             │        46,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 96)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 416)            │        40,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 416)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 160)            │        66,720 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 160)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         5,152 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 8)              │           264 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 162,504 (634.78 KB)

 Trainable params: 162,504 (634.78 KB)

 Non-trainable params: 0 (0.00 B)

In [16]:
# Save the best model to a file
lda_param = lda_best.get_config()
with open('search_nn/lda_param.json', 'w') as f:
    json.dump(lda_param, f)

In [17]:
for layer in lda_best.layers:
    config = layer.get_config()  # Get the layer's configuration dict
    # The 'activation' key in the config dict contains the activation function name
    if 'activation' in config:
        print(f"Layer: {layer.name}, Activation Function: {config['activation']}")
    else:
        print(f"Layer: {layer.name}, No activation function")

Layer: dense, Activation Function: relu
Layer: dropout, No activation function
Layer: dense_1, Activation Function: sigmoid
Layer: dropout_1, No activation function
Layer: dense_2, Activation Function: softplus
Layer: dropout_2, No activation function
Layer: dense_3, Activation Function: relu
Layer: dropout_3, No activation function
Layer: dense_4, Activation Function: relu
Layer: dropout_4, No activation function
Layer: dense_5, Activation Function: softmax


In [18]:
for layer in lda_best.layers:
    config = layer.get_config()  # Extract the layer configuration as a dictionary
    layer_type = config['name'].split('_')[0]  # Get the type of layer (e.g., "dense", "dropout")
    
    # Print layer type and configuration
    print(f"Layer Type: {layer_type.upper()}")
    if layer_type == 'dense':
        print(f"  - Units: {config['units']}")
        print(f"  - Activation: {config['activation']}")
    elif layer_type == 'dropout':
        print(f"  - Rate: {config['rate']}")

Layer Type: DENSE
  - Units: 480
  - Activation: relu
Layer Type: DROPOUT
  - Rate: 0.35000000000000003
Layer Type: DENSE
  - Units: 96
  - Activation: sigmoid
Layer Type: DROPOUT
  - Rate: 0.35000000000000003
Layer Type: DENSE
  - Units: 416
  - Activation: softplus
Layer Type: DROPOUT
  - Rate: 0.15000000000000002
Layer Type: DENSE
  - Units: 160
  - Activation: relu
Layer Type: DROPOUT
  - Rate: 0.15000000000000002
Layer Type: DENSE
  - Units: 32
  - Activation: relu
Layer Type: DROPOUT
  - Rate: 0.0
Layer Type: DENSE
  - Units: 8
  - Activation: softmax


In [19]:
test_loss, test_acc = lda_best.evaluate(X_val_lda, y_val_onehot)
print(f"Test Accuracy: {test_acc:.4f}")

38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5847 - loss: 1.2790 
Test Accuracy: 0.5625


### PCA

In [20]:
# Apply PCA for dimensionality reduction
p_PCA = 25 # from notebook pictures
pca = PCA(n_components=p_PCA, random_state=42)  # Select top 25 components
X_train_pca = pca.fit_transform(X_train_sc)
X_val_pca = pca.transform(X_val_sc)

In [21]:
def build_model_pca(hp):
    model = Sequential()
    model.add(Dense(units=hp.Int('units', min_value=32, max_value=518, step=32),
                    activation=hp.Choice('activation', values=['relu', 'softplus', 'tanh', 'sigmoid']),
                    input_shape=(p_PCA,)))  # Adjusted to match PCA output
    model.add(Dropout(rate=hp.Float('dropout_0', min_value=0.0, max_value=0.5, step=0.05)))

    for i in range(hp.Int('layers', 1, 5)):
        model.add(Dense(units=hp.Int(f'units_{i}', min_value=32, max_value=518, step=32),
                        activation=hp.Choice(f'activation_{i}', values=['relu', 'softplus', 'tanh', 'sigmoid'])))
        model.add(Dropout(rate=hp.Float(f'dropout_{i+1}',  min_value=0.0, max_value=0.5, step=0.05)))

    model.add(Dense(8, activation='softmax'))  # Assuming 8 classes, adjust as necessary
    model.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['accuracy'])
    return model


# Be careful with this operation to avoid deleting important data
shutil.rmtree('my_dir/hparam_tuning')

# Now, create a new tuner instance as before
tuner_pca = RandomSearch(
    build_model_pca,
    objective='val_accuracy',
    executions_per_trial=1,
    directory='my_dir',
    project_name='hparam_tuning'  # The same project name can be reused after deletion
)

c:\Users\Liu_h\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [22]:
tuner_pca.search(X_train_pca, y_train_onehot, epochs=20, validation_data=(X_val_pca, y_val_onehot))    

Trial 10 Complete [00h 00m 20s]
val_accuracy: 0.4975000023841858

Best val_accuracy So Far: 0.5224999785423279
Total elapsed time: 00h 02m 31s


In [23]:
# Get the best model
pca_best = tuner_pca.get_best_models(num_models=1)[0]
pca_best.summary()

c:\Users\Liu_h\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 26 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 256)            │         6,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 512)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 96)             │        49,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 96)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 416)            │        40,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 416)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 448)            │       186,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 448)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 8)              │         3,592 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 418,248 (1.60 MB)

 Trainable params: 418,248 (1.60 MB)

 Non-trainable params: 0 (0.00 B)

In [24]:
# Save the best model to a file
pca_param = pca_best.get_config()
with open('search_nn/pca_param.json', 'w') as f:
    json.dump(pca_param, f)

In [25]:
for layer in pca_best.layers:
    config = layer.get_config()  # Get the layer's configuration dict
    # The 'activation' key in the config dict contains the activation function name
    if 'activation' in config:
        print(f"Layer: {layer.name}, Activation Function: {config['activation']}")
    else:
        print(f"Layer: {layer.name}, No activation function")

Layer: dense, Activation Function: relu
Layer: dropout, No activation function
Layer: dense_1, Activation Function: tanh
Layer: dropout_1, No activation function
Layer: dense_2, Activation Function: tanh
Layer: dropout_2, No activation function
Layer: dense_3, Activation Function: relu
Layer: dropout_3, No activation function
Layer: dense_4, Activation Function: softplus
Layer: dropout_4, No activation function
Layer: dense_5, Activation Function: softmax


In [26]:
for layer in pca_best.layers:
    config = layer.get_config()  # Extract the layer configuration as a dictionary
    layer_type = config['name'].split('_')[0]  # Get the type of layer (e.g., "dense", "dropout")
    
    # Print layer type and configuration
    print(f"Layer Type: {layer_type.upper()}")
    if layer_type == 'dense':
        print(f"  - Units: {config['units']}")
        print(f"  - Activation: {config['activation']}")
    elif layer_type == 'dropout':
        print(f"  - Rate: {config['rate']}")

Layer Type: DENSE
  - Units: 256
  - Activation: relu
Layer Type: DROPOUT
  - Rate: 0.45
Layer Type: DENSE
  - Units: 512
  - Activation: tanh
Layer Type: DROPOUT
  - Rate: 0.35000000000000003
Layer Type: DENSE
  - Units: 96
  - Activation: tanh
Layer Type: DROPOUT
  - Rate: 0.2
Layer Type: DENSE
  - Units: 416
  - Activation: relu
Layer Type: DROPOUT
  - Rate: 0.0
Layer Type: DENSE
  - Units: 448
  - Activation: softplus
Layer Type: DROPOUT
  - Rate: 0.2
Layer Type: DENSE
  - Units: 8
  - Activation: softmax


In [27]:
test_loss, test_acc = pca_best.evaluate(X_val_pca, y_val_onehot)
print(f"Test Accuracy: {test_acc:.4f}")

38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5217 - loss: 1.4350 
Test Accuracy: 0.5225
